In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib as plt
from tensorflow.keras import layers
from keras.layers import Activation, Dense, LocallyConnected1D
from keras.layers import LocallyConnected2D, MaxPooling2D

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_df = pd.read_csv("/content/drive/MyDrive/train7291.csv")
test_df = pd.read_csv("/content/drive/MyDrive/test2007.csv")

In [4]:
# Converts into a numpy array
train_set = train_df.values
print(train_set.size)
test_set = test_df.values
print(test_set.size)

1873787
515799


In [5]:
# Separate labels and features
x_train = train_set[:,1:257]
x_test = test_set[:,1:257]

y_train = train_set[:,0]
y_test = test_set[:,0]

#convert labels -> int
y_test = y_test.astype(int)
y_train = y_train.astype(int)
print("Test labels: ", y_test)
print("Train labels: ", y_train)

Test labels:  [9 6 3 ... 4 0 1]
Train labels:  [6 5 4 ... 3 0 1]


In [6]:
# x_train: Dimension, shape, size
print(x_train.ndim)
x_train = x_train.reshape(7291, 16, 16)
print(x_train.shape)
print(x_train.size)

2
(7291, 16, 16)
1866496


In [7]:
# y_train: Shape, size, dimension
print(y_train.ndim)
print(y_train.shape)
print(y_train.size)

1
(7291,)
7291


In [8]:
#scale the images to the [0,1] range
x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32") / 255

In [9]:
# Change the image shape
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (7291, 16, 16, 1)
7291 train samples
2007 test samples


In [10]:
# convert class vectors to binary class matrices
num_classes = 10
input_shape = (16,16,1)

In [11]:
# Parameter initialization strategies
initializerOne = tf.keras.initializers.Constant(1.)
initializerTwo = tf.keras.initializers.Constant(0.5)
initializerThree = tf.keras.initializers.Constant(0.1)
initializerFour = tf.keras.initializers.Constant(5.)

# Learning rate optimizer/scheduler
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.92,
    staircase=True)

In [12]:
#CAUTION -> Running this twice will increase the dimension of y_test and y_train
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)

(7291, 10)


In [24]:
# model with zero wight initializer
'''
initializerZero = tf.keras.initializers.Zeros()

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.LocallyConnected2D(32, kernel_size=(3, 3), activation="relu", kernel_initializer=initializerZero),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.LocallyConnected2D(64, kernel_size=(3, 3), activation="tanh", kernel_initializer=initializerZero),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)


model.summary()'''

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
locally_connected2d_8 (Local (None, 14, 14, 32)        62720     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
locally_connected2d_9 (Local (None, 5, 5, 64)          462400    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)               

In [17]:
# model with orthogonal class wight initializer
'''
initializerOth = tf.keras.initializers.Orthogonal(gain=1.0, seed=None)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.LocallyConnected2D(32, kernel_size=(3, 3), activation="relu", kernel_initializer=initializerOth),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.LocallyConnected2D(64, kernel_size=(3, 3), activation="tanh", kernel_initializer=initializerOth),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

'''
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
locally_connected2d_2 (Local (None, 14, 14, 32)        62720     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
locally_connected2d_3 (Local (None, 5, 5, 64)          462400    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

In [20]:
# model with One values wight initializer
'''
initializerOnes = tf.keras.initializers.Ones()

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.LocallyConnected2D(32, kernel_size=(3, 3), activation="relu", kernel_initializer=initializerOnes),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.LocallyConnected2D(64, kernel_size=(3, 3), activation="tanh", kernel_initializer=initializerOnes),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()'''

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
locally_connected2d_4 (Local (None, 14, 14, 32)        62720     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
locally_connected2d_5 (Local (None, 5, 5, 64)          462400    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [22]:
# model with Random Normal values wight initializer
'''
initializerRanNorm = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.LocallyConnected2D(32, kernel_size=(3, 3), activation="relu", kernel_initializer=initializerRanNorm),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.LocallyConnected2D(64, kernel_size=(3, 3), activation="tanh", kernel_initializer=initializerRanNorm),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()'''

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
locally_connected2d_6 (Local (None, 14, 14, 32)        62720     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
locally_connected2d_7 (Local (None, 5, 5, 64)          462400    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [ ]:
# non Initialization, basic model

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.LocallyConnected2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.LocallyConnected2D(64, kernel_size=(3, 3), activation="tanh"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

In [ ]:
# Batch size testing for optimization 128, 64 and 10
# Uncomment the respective batch size for each test

'''
#batch_size = 128 
#batch_size = 64
#batch_size = 10
epochs = 15
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
'''

In [ ]:
# Learning rate optimization code block
# Uncomment each optimizer depending on the learning rate to test

'''
batch_size = 128
epochs = 15
#optimizer = keras.optimizers.Adam(lr=0.01)  # uncoment for First test learning rate = 0.01
#optimizer = keras.optimizers.Adam(lr=0.001) # uncoment for Second test learning rate = 0.001
#optimizer = keras.optimizers.Adam(lr=0.0001) # uncoment for Third test learning rate = 0.0001
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
'''


In [ ]:
# Mommentum optimization code block
# Uncomment each sgd depending on the mommentum to test

'''
batch_size = 128
epochs = 15

#sgd = tf.keras.optimizers.SGD(lr=0.0001, momentum=0.5, nesterov=True)
#sgd = tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9, nesterov=True)
#sgd = tf.keras.optimizers.SGD(lr=0.0001, momentum=0.99, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
'''


In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [38]:
score = model.evaluate(x_train, y_train, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.015274176374077797
Test accuracy: 0.9968454241752625
